In [1]:
from qchart import start_listener, listener_is_running
from qchart.client import DataSender
from datetime import datetime
import copy
import numpy as np

In [2]:
if not listener_is_running():
    start_listener()

starting qchart listener at c:\users\nikha\repos\qchart\listener_start.py ...
qchart listener successfully started.


In [7]:
def get_new_id():
    t = datetime.now().strftime('%Y-%M-%d_%H-%M-%S-%f')
    return f'test-nik-{t}'

data_structure = {
    'x_param': {
        'values': [],
        'unit': 'A'
    },
    'y_param': {
        'values': [],
        'unit': '',
        'axes': ['x_param']
    }
}

x_vals = np.linspace(3,4,100)
y_vals = np.random.rand(*x_vals.shape)

sndr = DataSender(get_new_id())

data = copy.deepcopy(data_structure)
data['x_param']['values'] = x_vals
data['y_param']['values'] = y_vals

sndr.send_data(data)